In [1]:
import json
import csv

In [3]:
with open("playersData.txt", "r", encoding="utf-8") as f:
    for line in f.readlines():
        playerInfo = eval(line)
        playerUrl = playerInfo["url"]
#         基本信息，字典
        playerBaseInfo = {**playerInfo["base_info"], **playerInfo["capability_data"]}
        playerName = playerBaseInfo["中文名"].strip()
#         比赛信息，列表['赛季', '俱乐部', '上场', '首发', '进球', '助攻', '黄牌', '红牌', '替补']
        playerMatchInfo = playerInfo["match_data"]
#         荣誉信息，列表[['土耳其足球甲级联赛 X 1', '2003/2004']]
        playerHonors = playerInfo["honor_data"]
#         转会信息，列表[['2018-01-26', '伊斯坦布尔', '安塔利亚体育']
        playerTransfer = playerInfo["transfer_data"]
#         伤病信息，列表[['安塔利亚体育', '内侧副韧带受伤', '2020-11-27 ~ 2020-12-18']
        playerInjured = playerInfo["injured_data"]
        
        playerData = []
        quintuple = [playerName, "url", playerUrl, "", ""]
        playerData.append(quintuple)
        
        for k, v in playerBaseInfo.items():
            if not v == "":
                quintuple = [playerName, k.strip(), v.strip(), "", ""]
                playerData.append(quintuple)
        
        first = True
        for listt in playerMatchInfo:
            if first:
                first = False
            else:
                time = listt[0].split("/")
                club = listt[1]
                if len(time) == 2:
                    quintuple = [playerName, "效力于", club, time[0], time[1]]
                else:
                    quintuple = [playerName, "效力于", club, time[0], time[0]]
        
        last = True
        for i in range(len(playerTransfer)-1, -1, -1):#逆序遍历
            if last:#最后一行要特殊对待，因为没有起始时间
                last = False
                club = playerTransfer[i][1].strip()#
                quintuple = [playerName, "效力于", club, playerBaseInfo["生日"], playerTransfer[i][0]]
                playerData.append(quintuple)
            elif i == 0:#第一行也要特殊对待，一个是因为没有结束时间，一个是因为可以抽取出来两条转会信息
                club = playerTransfer[i][1].strip()#从第一个队提取出正常信息
                playerData.append([playerName, "效力于", club, playerTransfer[i+1][0], playerTransfer[i][0]])
                club = playerTransfer[i][2]#从第二个队提取出没有结束时间的信息
                playerData.append([playerName, "效力于", club, playerTransfer[i][0], "至今"])
                
            else:#其他时间点正常对待
                club = playerTransfer[i][1].strip()#
                playerData.append([playerName, "效力于", club, playerTransfer[i+1][0], playerTransfer[i][0]])
        
        for listt in playerInjured:
            time = listt[2].split("~")
            if len(time) == 2:
                playerData.append([playerName, "伤病", listt[1], time[0].strip(), time[1].strip()])
            elif len(time) == 1:
                playerData.append([playerName, "伤病", listt[1], time[0].strip(), time[0].strip()])
            else:
                playerData.append([playerName, "伤病", listt[1], "", ""])
        
        for listt in playerHonors:
            time = listt[1].split("/")
            honor = listt[0].split("X")
            honor = honor[0].strip()
            if len(time) == 2:
                playerData.append([playerName, "荣誉", honor, time[0].strip(), time[1].strip()])
            elif len(time) == 1:
                playerData.append([playerName, "荣誉", honor, time[0].strip(), time[0].strip()])
            else:
                playerData.append([playerName, "荣誉", honor, "", ""])
                
        with open("playersQuintuple-1.csv", "a+", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerows(playerData)